##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pesos dispersos con poda estructural

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

Los pesos de su modelo con poda estructural para que sea disperso en un patrón específico pueden acelerar el tiempo de inferencia del modelo con las compatibilidades de los hardware adecuados.

En este tutorial se muestra cómo:

- Definir y entrenar un modelo en el conjunto de datos mnist con una dispersión estructural específica
- Convertir el modelo podado al formato tflite
- Visualizar la estructura de los pesos podados

Para obtener una descripción general de la técnica de poda para la optimización del modelo, consulte la [descripción general de la poda](https://www.tensorflow.org/model_optimization/guide/pruning). Para ver un tutorial sobre la poda de pesos general, consulte [Poda en Keras](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras).

## Poda estructural de pesos

La poda estructural pone en cero sistemáticamente los pesos del modelo al inicio del proceso de entrenamiento. Estas técnicas de poda se aplican a bloques regulares de pesos para acelerar la inferencia en los hardware compatibles, por ejemplo: agrupar pesos en el modelo en bloques de cuatro y poner a cero dos de esos pesos en cada bloque, lo que se conoce como reducción de *2 por 4*. Esta técnica se aplica solo a la última dimensión del tensor de peso para el modelo que convierte TensorFlow Lite. Por ejemplo, los pesos de las capas `Conv2D` en TensorFlow Lite tienen la estructura `[channel_out, height, width, channel_in]` y los pesos de las capas `Dense` tienen la estructura `[channel_out, channel_in]`. El patrón de dispersión se aplica a los pesos en la última dimensión: `channel_in`.

En comparación con la dispersión aleatoria, la dispersión estructurada generalmente tiene una precisión menor debido a una estructura restrictiva; sin embargo, puede reducir significativamente el tiempo de inferencia en el hardware compatible.

La poda se puede aplicar a un modelo junto con otras técnicas de compresión de modelos para obtener una mejor tasa de compresión. Consulte ejemplos de cuantización y agrupación en [técnica colaborativa de optimización](https://blog.tensorflow.org/2021/10/Collaborative-Optimizations.html) para obtener más detalles.

## Preparación

Prepare su entorno de desarrollo y sus datos.

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization
! pip install -q matplotlib

In [ ]:
import tensorflow as tf
from tensorflow import keras

import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

## Descargar y normalizar datos de imágenes del conjunto de datos [MNIST](https://www.tensorflow.org/datasets/catalog/mnist)

In [ ]:
# Load MNIST dataset.
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

## Definir los parámetros de poda estructural

Defina los parámetros de poda y especifíque el tipo de poda estructural. Establezca los parámetros para la poda en `(2, 4)`. Estos ajustes significan que en un bloque de cuatro elementos, al menos dos con la magnitud más baja se ponen a cero.

No es necesario configurar el parámetro `pruning_schedule`. De forma predeterminada, la máscara de poda se define en el primer paso y no se actualiza durante el entrenamiento.

In [ ]:
pruning_params_2_by_4 = {
    'sparsity_m_by_n': (2, 4),
}

Defina los parámetros para una poda aleatoria con una dispersión prevista del 50 %.

In [ ]:
pruning_params_sparsity_0_5 = {
    'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.5,
                                                              begin_step=0,
                                                              frequency=100)
}

Defina la arquitectura del modelo y especifique qué capas podar. La poda estructural se aplica en función de las capas del modelo que seleccione.

En el siguiente ejemplo, podamos solo algunas de las capas. Podamos la segunda capa `Conv2D` y la primera capa `Dense`.

Tenga en cuenta que la primera capa `Conv2D` no se puede podar estructuralmente. Para podarla estructuralmente, debe tener más de un canal de entrada. En su lugar, podamos la primera capa `Conv2D` con poda aleatoria.

In [ ]:
model = keras.Sequential([
    prune_low_magnitude(
        keras.layers.Conv2D(
            32, 5, padding='same', activation='relu',
            input_shape=(28, 28, 1),
            name="pruning_sparsity_0_5"),
        **pruning_params_sparsity_0_5),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    prune_low_magnitude(
        keras.layers.Conv2D(
            64, 5, padding='same',
            name="structural_pruning"),
        **pruning_params_2_by_4),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    keras.layers.Flatten(),
    prune_low_magnitude(
        keras.layers.Dense(
            1024, activation='relu',
            name="structural_pruning_dense"),
        **pruning_params_2_by_4),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Entrene y evalúe el modelo.

In [ ]:
batch_size = 128
epochs = 2

model.fit(
    train_images,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    verbose=0,
    callbacks=tfmot.sparsity.keras.UpdatePruningStep(),
    validation_split=0.1)

_, pruned_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Pruned test accuracy:', pruned_model_accuracy)

Elimine el contenedor de poda para que no se incluya en el modelo cuando lo convierta al formato TensorFlow Lite.

In [ ]:
model = tfmot.sparsity.keras.strip_pruning(model)

## Convertir modelo a formato tflite

In [ ]:
import tempfile

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

_, tflite_file = tempfile.mkstemp('.tflite')
print('Saved converted pruned model to:', tflite_file)
with open(tflite_file, 'wb') as f:
  f.write(tflite_model)

## Visualizar y comprobar pesos

Ahora visualice la estructura de pesos en la capa `Dense` podada con dispersión de 2 por 4. Extraiga los pesos del archivo tflite.

In [ ]:
# Load tflite file with the created pruned model
interpreter = tf.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()

details = interpreter.get_tensor_details()

# Weights of the dense layer that has been pruned.
tensor_name = 'structural_pruning_dense/MatMul'
detail = [x for x in details if tensor_name in x["name"]]

# We need the first layer.
tensor_data = interpreter.tensor(detail[0]["index"])()

Para verificar que seleccionamos la capa correcta que se podó, imprima la forma del tensor de peso.

In [ ]:
print(f"Shape of Dense layer is {tensor_data.shape}")

Ahora visualizamos la estructura de un pequeño subconjunto del tensor de peso. La estructura del tensor de peso es dispersa en la última dimensión, con el patrón `(2,4)`: dos de cuatro elementos son ceros. Para que la visualización sea más clara, reemplazamos todos los valores distintos de cero por unos.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# The value 24 is chosen for convenience.
width = height = 24

subset_values_to_display = tensor_data[0:height, 0:width]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(subset_values_to_display) > 0, val_ones, val_zeros)

Defina la función ayudante para dibujar líneas de separación para ver la estructura con claridad.

In [ ]:
def plot_separation_lines(height, width):

    block_size = [1, 4]

    # Add separation lines to the figure.
    num_hlines = int((height - 1) / block_size[0])
    num_vlines = int((width - 1) / block_size[1])
    line_y_pos = [y * block_size[0] for y in range(1, num_hlines + 1)]
    line_x_pos = [x * block_size[1] for x in range(1, num_vlines + 1)]

    for y_pos in line_y_pos:
        plt.plot([-0.5, width], [y_pos - 0.5 , y_pos - 0.5], color='w')

    for x_pos in line_x_pos:
        plt.plot([x_pos - 0.5, x_pos - 0.5], [-0.5, height], color='w')

Ahora visualice el subconjunto del tensor de peso.

In [ ]:
plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structural pruning for Dense layer")
plt.show()

Visualice los pesos de la capa `Conv2D`. La dispersión estructural se aplica en el último canal, similar a la capa `Dense`. Sólo la segunda capa `Conv2D` se poda estructuralmente como se señaló anteriormente.

In [ ]:
# Get weights of the convolutional layer that has been pruned with 2 by 4 sparsity.
tensor_name = 'structural_pruning/Conv2D'
detail = [x for x in details if tensor_name in x["name"]]
tensor_data = interpreter.tensor(detail[1]["index"])()
print(f"Shape of the weight tensor is {tensor_data.shape}")

De manera similar a los pesos de la capa `Dense`, la última dimensión del núcleo tiene una estructura (2, 4).

In [ ]:
weights_to_display = tf.reshape(tensor_data, [tf.reduce_prod(tensor_data.shape[:-1]), -1])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 1e-9, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structurally pruned weights for Conv2D layer")
plt.show()

Veamos cómo se ven esos pesos podados de forma aleatoria. Los extraemos y mostramos un subconjunto del tensor de peso.

In [ ]:
# Get weights of the convolutional layer that has been pruned with random pruning.
tensor_name = 'pruning_sparsity_0_5/Conv2D'
detail = [x for x in details if tensor_name in x["name"]]
tensor_data = interpreter.tensor(detail[0]["index"])()
print(f"Shape of the weight tensor is {tensor_data.shape}")

In [ ]:
weights_to_display = tf.reshape(tensor_data, [tensor_data.shape[0],tf.reduce_prod(tensor_data.shape[1:])])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 0, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Unstructed pruned weights for Conv2D layer")
plt.show()

El kit de herramientas de optimización de modelos de TensorFlow incluye un script de Python que se puede usar para verificar si las capas del modelo del archivo tflite proporcionadas tienen los pesos estructuralmente podados: [`check_sparsity_m_by_n.py`](https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/python/core/sparsity/keras/tools/check_sparsity_m_by_n.py). El siguiente comando demuestra cómo usar esta herramienta para comprobar la dispersión de 2 por 4 en un modelo específico.

In [ ]:
! python3 ./tensorflow_model_optimization/python/core/sparsity/keras/tools/check_sparsity_m_by_n.py --model_tflite=pruned_model.tflite --m_by_n=2,4
